In [1]:
%run base.ipynb

In [2]:
Jq_ = Kinematics.Jq
J_ = Kinematics.J

In [3]:
xd = Jq_@x_nb
ode_xdd = uv_body.body_forward_dynamics()

rhs = vertcat(xd, ode_xdd) #the complete ODE vector
f_rhs = Function('Odefunc', [m, W, B, r_g, r_b, I_o,
                           decoupled_added_m, coupled_added_m,
                           linear_dc, quadratic_dc,
                           x_nb, v_c, eul, uq, tau_b], [rhs])

## RETURNS vertcat(p_n ,uq , x_nb)
ode = f_rhs(ps.m, ps.W, ps.B, ps.rg, ps.rb, ps.Io, ps.added_m,
                       ps.coupl_added_m, ps.linear_dc, ps.quadratic_dc, x_nb, v_c, q2euler(uq), uq,  tau_b)

xS0 = vertcat(p_n, uq, x_nb)
ode_p = vertcat(dt, v_c)

# integrator to discretize the system
sys = {}
sys['x'] = xS0
sys['u'] = tau_b
sys['p'] = ode_p
sys['ode'] = ode*dt # Time scaling

intg = integrator('intg', 'rk', sys, 0, 1 , {'simplify':True, 'number_of_finite_elements':5})

res = intg(x0=xS0,u=tau_b, p=ode_p) #evaluate with symbols
x_next = res['xf']

x_next[3:7] = x_next[3:7]/sqrt(x_next[3:7].T@x_next[3:7])  #quaternions requires normalization

x_next[9] = if_else(x_next[2] < 0, 0,  x_next[9]) # if vehicle on surface, no more up motion
x_next[2] = if_else(x_next[2] < 0, 0,  x_next[2]) # if vehicle on surface, keep on surface and not go up

In [5]:
# Simplify API to (x,u)->(x_next)
F = Function('Vnext',[xS0, tau_b, dt, v_c],[x_next])

T = 10 # time horizon
N = 1600 # number of control intervals

In [5]:
# c , cpp or matlab code generation for forward dynamics
# F.generate("Vnext.c")
# os.system(f"gcc -fPIC -shared Vnext.c -o libVnext.so")

0